# Text Classification

텍스트 분류란 텍스트 형태의 데이터를 원하고자하는 카테고리로 구분하는 분석방법이다. 텍스트 데이터에는 다양한 종류가 있는데, 간단한 단어, 문장 부터 전체 문서까지 사람이 매일 제일 많이 생성해내는 데이터중에 하나 이다.

이번 실습에서는 영화 리뷰 데이터셋을 가지고 어떤 리뷰가 `positive` 인지 `negative` 인지 분류하는 모델을 구축하고자 한다.

## 데이터

### 데이터 불러오기

In [4]:
from nltk.corpus import movie_reviews

fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
label = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

print('리뷰 갯수:', len(reviews))
print('첫번째 리뷰 길이:', len(reviews[0]))
print('종속변수 레이블:', set(label))

리뷰 갯수: 2000
첫번째 리뷰 길이: 4043
종속변수 레이블: {'neg', 'pos'}


### train set과 test set으로 나누기

`scikit learn`의 `train_text_split`를 이용하면 손쉽게 train set과 test set으로 나눌 수 있다.

여기선 train set과 test set을 8:2로 나누었다.

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, label, test_size=0.2, random_state=10)
# sklearn의 train_test_split 함수는 먼저 data set을 shuffle하고 주어진 비율에 따라 train set과 test set을 나눠 줌
# 위에서는 reviews를 X_train과 X_test로 8:2의 비율로 나누고, label을 y_train과 y_test로 나눔
# 이 때 X와 y의 순서는 동일하게 유지해서 각 입력값과 label이 정확하게 match되도록 함
# random_state는 shuffle에서의 seed 값으로, 지정한 경우 항상 동일한 결과로 shuffle이 됨

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


### TFIDF로 데이터 변환하기

TFIDF는 Term Frequency - Inverse Document Frequency의 약자로 단어의 빈도와 역 문서 빈도를 사용하여 DTM(Document Term Matrix) 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법이다.

TFIDF도 `scikit learn` 라이브러리를 이용해 만들 수 있다.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer().fit(X_train) # X_train을 이용하여 vectorizer를 학습
tfidf #vectorize에서 사용한 매개변수 값들을 확인 -> 현재는 모두 default 값을 사용, 향후 tokenizer, max_features 등을 지정할 수 있음

TfidfVectorizer()

In [23]:
X_train_tfidf = tfidf.transform(X_train) #학습된 vectorizer를 이용하여 train set을 변환
X_train_tfidf.shape # 1600 (review 수) x 36310 (전체 corpus에서 사용된 단어의 수) 크기로 vector set이 생성됨
# matrix 안의 값은 해당 tfidf score임

(1600, 36310)

사용된 단어의 수가 너무 많은 경우 `TfidfVectorizer`의 `max_features` 파라미터값을 조정할 수 있다.

In [24]:
tfidf = TfidfVectorizer(max_features=2000).fit(X_train)
tfidf

TfidfVectorizer(max_features=2000)

이제 train set과 test set을 변환시켜보자.

In [25]:
X_train_tfidf = tfidf.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_tfidf.shape) # 36310 대신 2000이 된 것을 확인
X_test_tfidf = tfidf.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_tfidf.shape)

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


`X_train_tfidf`가 어떻게 생겼는지 한번 확인해보자.

In [27]:
import pandas as pd

df = pd.DataFrame(X_train_tfidf.toarray())
df

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.03273,...,0.000000,0.000000,0.0,0.000000,0.052729,0.000000,0.0,0.018282,0.000000,0.0
1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.028947,0.032614,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.049245,0.0,0.000000,0.059593,0.000000,0.0,0.000000,0.059377,0.0
3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.021087,0.0,0.000000,0.000000,0.0
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.081726,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.0,0.320001,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.026472,0.000000,0.0,0.000000,0.127176,0.000000,0.0,0.055118,0.000000,0.0
1596,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.076539,0.000000,0.0,0.033172,0.000000,0.0
1597,0.0,0.024786,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.016403,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1598,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.03884,0.00000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.023814,0.0,0.000000,0.000000,0.0


## 분류 기법

이제 데이터가 준비되었으니 모델을 구축해보자. 여러 가지 분류 기법들이 있지만 여기선 **나이브 베이즈 분류**와 **로지스틱 회귀 분석** 기법을 사용하여 모델을 구축한다.

두가지 모두 `scikit learn`에서 손쉽게 모델을 구축할 수 있다.

### Naive Bayes Classification

**나이브 베이즈 분류(Naïve Bayes Classification)** 는 특성들 사이의 독립을 가정하는 베이즈 정리를 적용한 확률 분류기이다. 또한 나이브 베이즈 분류는 텍스트 분류에 사용됨으로써 문서를 여러 범주 (예: 스팸, 스포츠, 정치)중 하나로 판단하는 문제에 대한 대중적인 방법으로 남아있다.

나이브 베이즈 분류는 TFIDF를 사용하지 않고 단어의 빈도수를 사용한다. 따라서 `scikit learn`의 `CountVectorizer`을 사용한다.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


`X_train_cv`이 어떻게 생겼는지 확인해 보자.

In [18]:
tokens = cv.get_feature_names_out()
df = pd.DataFrame(X_train_cv.toarray(), columns=tokens)
df

,000,10,100,13,15,1995,1996,1997,1998,1999,...,years,yes,yet,york,you,young,younger,your,yourself,zero
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,5,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,3,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,3,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0,8,0,0,0,0,0,0,0,0,...,1,0,0,0,8,0,0,2,0,0
1596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,0,0,1,0,0
1597,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1598,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


이제 모델을 구축해보자. 

실제로 필요한 것은 test set에 대한 예측정확도이나, 과적합 (overfitting)의 문제가 있는지를 보기 위해 train set에 대한 예측정확도를 같이 확인해본다.

In [29]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB() # 분류기 선언
NB_clf.fit(X_train_cv, y_train) #train set을 이용하여 분류기(classifier)를 학습

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) #test set에 대한 예측정확도를 확인

Train set score: 0.864
Test set score: 0.775


train set에 대한 예측정확도는 86.4% 이고 test set에 대한 예측정확도는 77.5% 이다.

### Logistic Regression

 로지스틱 회귀는 독립 변수의 선형 결합을 이용하여 사건의 발생 가능성을 예측하는데 사용되는 통계 기법이다. 선형 회귀 분석과는 다르게 종속 변수가 범주형 데이터를 대상으로 하며 입력 데이터가 주어졌을 때 해당 데이터의 결과가 특정 분류로 나뉘기 때문에 일종의 분류 (classification) 기법으로도 볼 수 있다.

 데이터를 TFIDF로 변환해서 모델을 구축한것과 Count Vector로 변환해서 모델을 구축한것으로 로지스틱 회귀분석을 했을 때 정확도를 비교해본다.

In [30]:
from sklearn.linear_model import LogisticRegression

LR_clf_cv = LogisticRegression()
LR_clf_cv.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(LR_clf_cv.score(X_train_cv, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf_cv.score(X_test_cv, y_test))) # test data에 대한 예측정확도

# count vector를 이용한 regression 결과가 tfidf보다 더 좋게 나옴
# 보통은 tfidf가 더 좋은 결과를 보이는데, 이와 같이 상황에 따라 다른 결과가 나오기도 함
# 지금은 train data의 수가 1,600개인데 비해, 추정해야 하는 parameter의 수가 2,000개로 sample 수가 학습에 부족한 상황, 
# 따라서 예상 못한 다양한 결과가 나올 수 있음
# 좀더 data가 많은 상황에서의 test가 필요

Train set score: 1.000
Test set score: 0.823


/Users/cho2jiwoo/docs/study/Python/Notebook/notebook/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Count Vector를 사용하여 구축한 모델의 train set에 대한 예측정확도는 100% 이고 test set에 대한 예측정확도는 82.3% 이다.

근데 지금은 train data의 수가 1,600개인데 비해, 추정해야 하는 parameter의 수가 2,000개로 sample 수가 학습에 부족한 상황이라 경고 메세지가 떴다.

In [31]:
LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

Train set score: 0.917
Test set score: 0.820


TFIDF를 사용하여 구축한 모델의 train set에 대한 예측정확도는 91.7% 이고 test set에 대한 예측정확도는 82.0% 이다.

## 텍스트 마이닝의 문제점

텍스트 마이닝은 텍스트를 연산 가능한 수치로 변환해야 하기 때문에 일반적인 수치 데이터들을 분석할 때 와는 다른 문제점들이 있다. 기본적인것 몇개만 소개하고자 한다.

**Curse of dimensionality**

차원의 저주라고 불리는 curse of dimensionality 문제는 텍스트를 벡터화 시켰을 때 생기는 차원이 매우 커서 학습할 데이터의 수보다 커지며 발생한다. 그리고 sparse data일 수록 차원은 더욱 커지며 이렇게 학습된 모델은 성능이 좋지 않아진다. 차원의 저주를 해결하기 위해선 더 많은 데이터를 학습시키거나 차원을 축소(dimension reduction)하는 과정을 거쳐야 한다.

**단어 빈도의 불균형**

문서에서 단어별 빈도수는 long right tail 형태를 띄게 된다. 많이 나타나는 단어들은 많은 정보를 담고 있지 않을 경우가 많다.

**단어가 쓰인 순서 정보의 손실**

정통적인 BOW 나 TFIDF는 문서를 단어들로 쪼개어 벡터로 만들었다. 그렇게 되면 문서가 가지고 있는 문맥을 잃게 되어 문장의 의미를 파악하기 어려워진다.

여기선 차원의 저주 문제를 해결하기 위한 여러 기법들을 살펴보려 한다.

## Feature Extraction

**정규화 (Regularization)**: 회귀모델에서의 정규화란 모델을 변형하여 과적합을 완화해 일반화 성능을 높여주기 위한 기법을 말한다.

두가지 정규화 기법이 있다. **Ridge Regression**과 **Lasso Regression**.

### Ridge Regression

릿지 회귀는 기존 다중회귀선을 훈련데이터에 덜 적합하도록 만들어서 더 좋은 모델이 만들어지도록 하는 것이다. 과적합을 줄이기 위해서사용되는 회귀방법으로 모델의 복잡도를 줄이는 방법을 사용하는데 feature의 수를 줄이거나 모델을 단순한 모양으로 적합하는 것이다.

릿지 회귀는 편향(bias)를 조금 더하고, 분산(variance)를 줄이는 방법으로 정규화(regulation)을 수행한다.

일반적으로 ridge regression을 쓰면 쓰지 않은 경우보다 train data에 대한 예측정확도는 떨어지고 test data는 올라가게 되는데 이번 실습 예제에선 둘다 상승하는것을 볼 수 있었다.

In [32]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.974
Test set score: 0.838


### Lasso Regression

기존의 선형 회귀에서는 적절한 가중치와 편향을 찾아내는 것이 관건이었다. 라쏘는 거기에 덧붙여서 추가 제약 조건을 준다. 어떠한 제약을 줬냐하면, MSE가 최소가 되게 하는 가중치와 편향을 찾는데 동시에 가중치들의 절대값들의 합, 즉 가중치의 절대값들이 최소(기울기가 작아지도록)가 되게 해야한다는 것이다. 다시 말해서 가중치의 모든 원소가 0이 되거나 0에 가깝게 되게 해야한다. 따라서 어떤 특성들은 모델을 만들때 사용되지 않는다. 

In [33]:
from sklearn.linear_model import LogisticRegression
import numpy as np

lasso_clf = LogisticRegression(penalty='l1', solver='liblinear') # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))

# parameter 혹은 coefficient 중에서 0이 아닌 것들의 개수를 출력
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 

Train set score: 0.804
Test set score: 0.770
Used features count: 78 out of 2000


2000개 중에서 78개만 선택된 것을 볼 수 있다.

예측률은 rigde나 일반 logistic에 비해 떨어지지만, 실제로 영향을 미치는 단어들이 어떤 것들인지 확인할 수 있다는 장점이 있다.

In [35]:
print(len(tfidf.vocabulary_)) # tfidf에 사용된 단어의 수
tfidf_voca = tfidf.get_feature_names_out() # tfidf에서 단어이름을 가져옴
tfidf_voca[:10] # 앞 10개를 출력

2000


array(['000', '10', '100', '13', '15', '1995', '1996', '1997', '1998',
       '1999'], dtype=object)

 그럼 어떤 단어들이 선택되었는지 확인해보자.

In [36]:
selected_features = []
for i, sign in enumerate((lasso_clf.coef_ != 0)[0].tolist()):
    if sign: selected_features.append(tfidf_voca[i]) #사용여부가 True인 단어들만 selected_features에 저장

print(f'선택된 단어 갯수: {len(selected_features)}')
print(selected_features)

선택된 단어 갯수: 78
['aliens', 'also', 'and', 'any', 'as', 'attempt', 'awful', 'bad', 'batman', 'boring', 'cameron', 'carpenter', 'could', 'definitely', 'director', 'dull', 'even', 'excellent', 'family', 'great', 'hanks', 'harry', 'have', 'he', 'her', 'here', 'his', 'in', 'is', 'jackie', 'julie', 'lame', 'least', 'lebowski', 'life', 'looks', 'many', 'matrix', 'mess', 'most', 'movie', 'mulan', 'no', 'nothing', 'off', 'on', 'only', 'perfect', 'perfectly', 'plot', 'political', 'poor', 'quite', 'reason', 'ridiculous', 'scream', 'script', 'seagal', 'see', 'seen', 'shrek', 'so', 'stupid', 'supposed', 'the', 'then', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'waste', 'well', 'west', 'why', 'worst']


## Feature Selection

Feature Selection의 종류:
- Principal Component Analysis
- Latent Semantic Analysis
- Singular-Value Decomposition

### Princiapl Component Analysis

주성분 분석 이라고도 하며 여러 feature들 중에서 데이터 집합을 가장 잘 나타내는 성분을 추출해내는 방법이다.

PCA는 수학적으로 직교 선형 변환으로 정의된다. 직교 선형 변환이란 어떤 데이터를 새로운 좌표계로 변환하는 것을 말한다. 데이터를 투사했을 때 가장 큰 분산이 첫 번째 좌표로 오고, 그 다음으로 큰 분산이 두 번째 좌표로 오고, 그 다음이 그 다음 좌표로 오는 식으로 할 수 있다. 여기서 첫 번째 좌표를 첫 번째 주성분이라고 부른다.

### Latent Semantic Analysis

$$
A = U\Sigma V^T
$$

잠재 의미 분석 이라고도 불리며 단어-문서 행렬, 단어-문맥 행렬 등 입력 데이터를 특이값 분해(singular-value decomposition)하여 차원수를 줄여 계산을 효율화하고 행간의 숨은 의미를 도출하는 분석이다. 단어 사용빈도 등 말뭉치의 통계량 정보가 들어 있는 행렬에 특이값 분해 등 수학적 기법을 적용해 행렬에 속한 벡터들의 차원을 축소하는 방법이다.

In [37]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  # train data로 학습 -> unsupervised이므로 y가 필요 없음
print(f'계산된 각 component가 설명하는 분산의 비율\n{svd.explained_variance_ratio_}')
print(f'선택된 component들이 설명하는 분산의 합\n{svd.explained_variance_ratio_.sum()}') # 선택한 component의 수에 따라 달라짐

계산된 각 component가 설명하는 분산의 비율
[0.01525261 0.01669138 0.0133583  0.01042773 0.00741403 0.00730082
 0.00677966 0.00583257 0.00555167 0.00520324 0.00493018 0.00489498
 0.00458197 0.0044574  0.00432849 0.00421975 0.0041921  0.00413251
 0.00403416 0.00398954 0.00388074 0.00379435 0.00377392 0.00370049
 0.00361687 0.00358558 0.00350955 0.00340951 0.0033149  0.00327589
 0.00321078 0.00319169 0.00315556 0.00311055 0.00304756 0.00302892
 0.00300949 0.00297231 0.00293611 0.00292412 0.00290208 0.00287548
 0.0028342  0.00280742 0.00276097 0.00274277 0.00270645 0.00268513
 0.00267427 0.00265801 0.00258256 0.00257419 0.00256683 0.00255658
 0.00251155 0.00250159 0.00248046 0.00247207 0.00245957 0.00245289
 0.00241614 0.0023879  0.00236025 0.00234094 0.00233893 0.00231345
 0.00229516 0.0022807  0.00227705 0.00224271 0.00222958 0.00222203
 0.00220312 0.00219438 0.00218776 0.0021722  0.00215608 0.00215051
 0.00213564 0.00212784 0.00211271 0.00208695 0.00207613 0.00206077
 0.00204613 0.00203201 0.00200931

현재 결과에서는 100개의 선택된 component가 전체 분산의 34% 정도를 설명하고 있음

![](https://www.fun-coding.org/00_Images/SVD.png)
출처: https://www.fun-coding.org/recommend_basic6.html

In [39]:
print(f'특이값\n{svd.singular_values_}')
print(f'우특이벡터\n{svd.components_.shape}')

특이값
[26.28071313  3.92896916  3.51203674  3.10301032  2.61748589  2.59642773
  2.50217867  2.32098855  2.26439245  2.19203943  2.13375967  2.12626195
  2.0574806   2.02899959  1.99946031  1.97402069  1.96747448  1.95357853
  1.93008885  1.91932804  1.89298024  1.87190046  1.86671996  1.84849394
  1.82766938  1.819579    1.80027893  1.77431404  1.74951541  1.7391934
  1.72195946  1.71670307  1.70696488  1.69474953  1.67750176  1.67234783
  1.66697967  1.65665295  1.64652842  1.64316809  1.63696717  1.62944089
  1.61771724  1.61004634  1.59668252  1.59144373  1.58090069  1.57474131
  1.57139547  1.56660944  1.54422075  1.54187887  1.53957662  1.53645066
  1.52294988  1.51981377  1.5133942   1.51083776  1.50699561  1.50498852
  1.4936319   1.48487743  1.47627146  1.47043492  1.4695741   1.461563
  1.45597621  1.45119181  1.4500094   1.43903939  1.43480772  1.43237878
  1.42628258  1.42346676  1.4212888   1.41623469  1.41104021  1.40916811
  1.4042597   1.40182779  1.39671277  1.38815761  

LSA를 이용하여 로지스틱 회귀 모델을 구축해 보자.

In [41]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
print(X_train_tfidf.shape) #축소하기 전의 차원
print(X_train_svd.shape) #축소된 후의 차원, 2000 -> 100개로 줄어 있음

X_test_svd = svd.transform(X_test_tfidf)
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train) #축소된 값으로 학습
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

(1600, 2000)
(1600, 100)
Train set score: 0.838
Test set score: 0.790


train set에 대한 예측정확도는 83.8% 이고 test set에 대한 예측정확도는 79.0% 이다.

나이브베이즈분류기, Lasso를 사용한 모델 보다 좋지만 Ridge, 일반 모델 보다는 안좋은 성능을 보여준다.

## 모델 비교

지금까지 구축한 모델들의 성능을 비교해보자.

| 모델 | Train set | Test set |
|-----|-----------|----------|
| 나이브베이즈분류기 | 0.864 | 0.775 |
| 로지스틱회귀(Count Vector) | 1.00 | 0.823 |
| 로지스틱회귀(TFIDF) | 0.917 | 0.820 |
| 릿지 회귀 | 0.974 | 0.838 |
| 라쏘 회귀 | 0.804 | 0.77 |
| 로지스틱회귀(LSA) | 0.838 | 0.79 |

## Reference

- https://ko.wikipedia.org/wiki/나이브_베이즈_분류
- https://ko.wikipedia.org/wiki/로지스틱_회귀
- https://velog.io/@yuns_u/Ridge-Regression
- https://bskyvision.com/193
- https://ko.wikipedia.org/wiki/주성분_분석
- https://zetawiki.com/wiki/잠재_의미_분석